In [3]:
import os, argparse
import tensorflow as tf

def freeze_graph(model_dir, output_node_names):
    """Extract the sub graph defined by the output nodes and convert 
    all its variables into constant 
    Args:
        model_dir: the root folder containing the checkpoint state file
        output_node_names: a string, containing all the output node's names, 
                            comma separated
    """
    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    if not output_node_names:
        print("You need to supply the name of a node to --output_node_names.")
        return -1

    # We retrieve our checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    # We precise the file fullname of our freezed graph
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"

    # We clear devices to allow TensorFlow to control on which device it will load operations
    clear_devices = True

    # We start a session using a temporary fresh Graph
    with tf.Session(graph=tf.Graph()) as sess:
        # We import the meta graph in the current default Graph
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

        # We restore the weights
        saver.restore(sess, input_checkpoint)
        
        graph = tf.get_default_graph() 
        #print("graph: " + str(graph))
        
        #for n in tf.get_default_graph().as_graph_def().node:
        #    print("n.name : " + str(n.name))
        
        # We use a built-in TF helper to export variables to constants
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
            output_node_names.split(",") # The output node names are used to select the usefull nodes
        ) 

        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
            
        print("%d ops in the final graph." % len(output_graph_def.node))

    return output_graph_def

if __name__ == '__main__':
    #parser = argparse.ArgumentParser()
    #parser.add_argument("--model_dir", type=str, default="/home/kimbring2/catkin_ws/src/jetbot/jetbot_control/src/drqn", help="Model folder to export")
    #parser.add_argument("--output_node_names", type=str, default="jetbot_frozen_model", help="The name of the output nodes, comma separated.")
    #args = parser.parse_args()
    
    model_dir = "/home/kimbring2/catkin_ws/src/jetbot/jetbot_control/src/drqn"
    output_node_names = "main/ArgMax"
    freeze_graph(model_dir, output_node_names)

INFO:tensorflow:Restoring parameters from /home/kimbring2/catkin_ws/src/jetbot/jetbot_control/src/drqn/model-2.cptk
n.name : main/scalarInput
n.name : main/Reshape/shape
n.name : main/Reshape
n.name : main/Conv/weights/Initializer/random_uniform/shape
n.name : main/Conv/weights/Initializer/random_uniform/min
n.name : main/Conv/weights/Initializer/random_uniform/max
n.name : main/Conv/weights/Initializer/random_uniform/RandomUniform
n.name : main/Conv/weights/Initializer/random_uniform/sub
n.name : main/Conv/weights/Initializer/random_uniform/mul
n.name : main/Conv/weights/Initializer/random_uniform
n.name : main/Conv/weights
n.name : main/Conv/weights/Assign
n.name : main/Conv/weights/read
n.name : main/Conv/dilation_rate
n.name : main/Conv/Conv2D
n.name : main/Conv/Relu
n.name : main/Conv_1/weights/Initializer/random_uniform/shape
n.name : main/Conv_1/weights/Initializer/random_uniform/min
n.name : main/Conv_1/weights/Initializer/random_uniform/max
n.name : main/Conv_1/weights/Initial

INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.
168 ops in the final graph.


In [1]:
import tensorflow as tf
import numpy as np

def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
        
    return graph

if __name__ == '__main__':
    # Let's allow the user to pass the filename as an argument
    frozen_model_filename = "/home/kimbring2/catkin_ws/src/jetbot/jetbot_control/src/drqn/frozen_model.pb"
    
    # We use our "load_graph" function
    graph = load_graph(frozen_model_filename)

    # We can verify that we can access the list of operations in the graph
    #for op in graph.get_operations():
    #    print(op.name)
        
    # We access the input and output nodes 
    x = graph.get_tensor_by_name('prefix/main/scalarInput:0')
    y = graph.get_tensor_by_name('prefix/main/ArgMax:0')
    trainLength = graph.get_tensor_by_name('prefix/main/trainLength:0')
    batch_size = graph.get_tensor_by_name('prefix/main/batch_size:0')
    
    input_test = np.empty([84, 84, 3], dtype=float)
    input_test_flatten = input_test.flatten()
    
    # We launch a Session
    with tf.Session(graph=graph) as sess:
        # Note: we don't nee to initialize/restore anything
        # There is no Variables in this graph, only hardcoded constants 
        y_out = sess.run(y, feed_dict={
            x: [input_test_flatten],
            trainLength: 1,
            batch_size: 1
        })
        # I taught a neural net to recognise when a sum of numbers is bigger than 45
        # it should return False in this case
        print(y_out) # [[ False ]] Yay, it works!

[4]
